In [1]:
import os

In [2]:
pwd

'c:\\Users\\junu\\OneDrive\\Desktop\\ds\\MLOPS\\media_campign_cost_prediction\\branches\\basic-set-up\\Media-Campign-cost-prediction\\research'

In [3]:
os.chdir('../')

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class PredictionConfig:
    model_path:  Path

In [27]:
from src.constants import *
from src.utils.common import read_yaml,create_directories,save_model

class Configuration:
    def __init__(self,config_path=CONFIG_FILE_PATH
                      ,params_path=PARAMS_FILE_PATH
                      ,schema_path=SCHEMA_FILE_PATH) -> None:
        self.config=read_yaml(config_path)
        self.params=read_yaml(params_path)
        self.schema=read_yaml(schema_path)

        create_directories([self.config.artifacts_root])

    def get_prediction_config(self):
        config=self.config.prediction
        prediction_config=PredictionConfig(
            model_path=Path(config.model_path)
        )
        return prediction_config


In [80]:
import pandas as  pd
# from src.components.data_transformation import DataTransformation
from src.utils.common import load_model

class Prediction:
    def __init__(self,config: PredictionConfig) -> None:
        self.config=config

    def initiate_prediction(self,data: pd.DataFrame):
        transformed_data=DataTransformation.get_basic_transfomed_data(data=data)
        print(transformed_data.columns)

        model=load_model(self.config.model_path)

        prediction=model.predict(transformed_data)
        
        

        

In [77]:
#for one basic data

data=pd.read_csv('artifacts/data_ingestion/data.csv')
data=data.drop('cost',axis=1).iloc[0:1]
data=pd.DataFrame(data)



In [78]:
data

,id,store_sales(in millions),unit_sales(in millions),total_children,num_children_at_home,avg_cars_at home(approx).1,gross_weight,recyclable_package,low_fat,units_per_case,store_sqft,coffee_bar,video_store,salad_bar,prepared_food,florist
0,0,8.61,3.0,2.0,2.0,2.0,10.3,1.0,0.0,32.0,36509.0,0.0,0.0,0.0,0.0,0.0


In [63]:
type(data)

pandas.core.frame.DataFrame

In [81]:
config=Configuration()
pred_config=config.get_prediction_config()
pred=Prediction(pred_config)
prediction=pred.initiate_prediction(data=data)

[2024-04-06 20:36:59,864: INFO: 19 : call: file loaded succesfully from the path config\config.yaml]
[2024-04-06 20:36:59,872: INFO: 19 : call: file loaded succesfully from the path params.yaml]
[2024-04-06 20:36:59,874: INFO: 19 : call: file loaded succesfully from the path schema.yaml]
Index(['store_sales(in millions)', 'unit_sales(in millions)', 'total_children',
       'num_children_at_home', 'avg_cars_at home(approx).1', 'gross_weight',
       'recyclable_package', 'low_fat', 'units_per_case', 'store_sqft',
       'coffee_bar', 'video_store', 'salad_bar', 'prepared_food', 'florist'],
      dtype='object')


ValueError: feature_names mismatch: ['Unnamed: 0', 'store_sales(in millions)', 'unit_sales(in millions)', 'total_children', 'num_children_at_home', 'avg_cars_at home(approx).1', 'gross_weight', 'recyclable_package', 'low_fat', 'units_per_case', 'store_sqft', 'coffee_bar', 'video_store', 'salad_bar', 'prepared_food', 'florist'] ['store_sales(in millions)', 'unit_sales(in millions)', 'total_children', 'num_children_at_home', 'avg_cars_at home(approx).1', 'gross_weight', 'recyclable_package', 'low_fat', 'units_per_case', 'store_sqft', 'coffee_bar', 'video_store', 'salad_bar', 'prepared_food', 'florist']
expected Unnamed: 0 in input data

In [66]:
import pandas as pd
from sklearn.model_selection import train_test_split
from src.excption.exception import customexception
import sys
from src import logger
from src.utils.common import create_directories
from src.entity.config_entity import DataTransformationConfig
import os

class DataTransformation:
    def __init__(self,data_txm_config:DataTransformationConfig) -> None:
        self.config=data_txm_config

    

        

    def data_transformation_for_training(self):
        try:
            logger.info(f"Data transformation has started")
            data=pd.read_csv(self.config.data_path)
            data=self.get_basic_transfomed_data(data)
            train,test = train_test_split(data, test_size=0.3, random_state=42)
            logger.info(f"train data shape{train.shape} and test data shape {test.shape}")
            create_directories([self.config.data_transformation_root])
            train.to_csv(os.path.join(self.config.data_transformation_root,'train.csv'))
            test.to_csv(os.path.join(self.config.data_transformation_root,'test.csv'))
            logger.info(f"data transformation completed successfully!")
        except Exception as e:
            raise customexception(e,sys)

    
    def data_transforamtion_for_prediction(self,data :pd.DataFrame):
        data=self.get_basic_transfomed_data(data)
        return data
    
    @staticmethod
    def get_basic_transfomed_data(data: pd.DataFrame):
        if 'id' in data.columns:
            data=data.drop('id',axis=1)
        data=data.dropna()
        data=data.drop_duplicates()
        return data








In [65]:
# from src.components.data_transformation import DataTransformation

d=DataTransformation.get_basic_transfomed_data(data)
d

,store_sales(in millions),unit_sales(in millions),total_children,num_children_at_home,avg_cars_at home(approx).1,gross_weight,recyclable_package,low_fat,units_per_case,store_sqft,coffee_bar,video_store,salad_bar,prepared_food,florist
0,8.61,3.0,2.0,2.0,2.0,10.3,1.0,0.0,32.0,36509.0,0.0,0.0,0.0,0.0,0.0


In [72]:
if 'id' in data.columns:
        data=data.drop('id',axis=1)
data=data.dropna()
data=data.drop_duplicates()

In [74]:
data.columns

Index(['store_sales(in millions)', 'unit_sales(in millions)', 'total_children',
       'num_children_at_home', 'avg_cars_at home(approx).1', 'gross_weight',
       'recyclable_package', 'low_fat', 'units_per_case', 'store_sqft',
       'coffee_bar', 'video_store', 'salad_bar', 'prepared_food', 'florist'],
      dtype='object')

In [52]:
data.drop('id',axis=1)

KeyError: "['id'] not found in axis"

In [53]:
d

,1
id,1.00
store_sales(in millions),5.00
unit_sales(in millions),2.00
total_children,4.00
num_children_at_home,0.00
avg_cars_at home(approx).1,3.00
gross_weight,6.66
store_sqft,28206.00


In [41]:
class Family:
    def __init__(self,var) -> None:
        self.var=var
    
    @staticmethod
    def name(name):
        print(name)
        return name

name=Family.name('junu')
print(name)





junu
junu
